# Generate experimental features -  experiments - cell1_211006_3148

In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import MEAutility as mu
from pprint import pprint
import time
import numpy as np
import neuroplotlib as nplt

import multimodalfitting as mf

import sys
import shutil

from pathlib import Path
import os
from datetime import datetime

%matplotlib notebook

In [ ]:
save_fig = False

In [ ]:
base_folder = Path("../../..")

In [ ]:
cell_name = "cell1_211006_3148"  # "cell1_211006_3148" | "cell1_211011_3436"

In [ ]:
data_folder = base_folder / "experimental_data" / cell_name

cell_models_folder = base_folder / "cell_models"
model_folder = cell_models_folder / cell_name
model_name = cell_name
fitting_folder = model_folder / "fitting"
extra_folder = data_folder / "extracellular"

## 1) Generate features and protocols with BluePyEfe

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

In [ ]:
with (extra_folder / "probe.json").open() as f:
    probe_info = json.load(f)
    probe_info["pitch"] = 17.5
    probe = mu.return_mea(info=probe_info)

In [ ]:
eap = np.load(extra_folder / "template.npy")
locations = np.load(extra_folder / "locations.npy")
fs = 20000

In [ ]:
ax_eap = mu.plot_mea_recording(eap, probe)

In [ ]:
efeatures_output_directory = fitting_folder / "efeatures"
ephys_dir = data_folder / "patch_data"

In [ ]:
from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    group_efeatures, create_feature_protocol_files, convert_legacy_targets
from bluepyefe.plotting import plot_all_recordings_efeatures

In [ ]:
from multimodalfitting.efeatures_extraction import build_wcp_metadata, wcp_reader, get_ecode_targets

In [ ]:
# select files for different repetitions

In [ ]:
# define timings for this experiment
ecodes_cell_timings = {
    "IDthres": {
        'ton': 400,
        'toff': 670
    },
    "firepattern": {
        'ton': 500,
        'toff': 4100
    },
    "IV": {
        'ton': 400,
        'toff': 3400
    },
    "IDrest": {
        'ton': 400,
        'toff': 1750
    },
    "APWaveform": {
        'ton': 350,
        'toff': 400
    },
    "HyperDepol": {
        'ton': 400,
        'toff': 1120,
        'tmid': 850
    },
    "sAHP": {
        'ton': 400,
        'toff': 1325,
        'tmid': 650,
        'tmid2': 875
    },
    "PosCheops": {
        'ton': 1000,
        't1': 9000,
        't2': 10500,
        't3': 14500,
        't4': 16000,
        'toff': 18660
    }
}

In [ ]:
if cell_name == "cell1_211006_3148":
    runs = [1, 2, 3, 4, 5]  # run1 --> different rheobase
elif cell_name == "cell1_211011_3436":
    runs = [3, 4, 5, 6]
    
ecode_names = list(ecodes_cell_timings.keys())

files_list = []

for run in runs:
    rep_dict = {}
    for ecode in ecode_names:
        for patch_file in ephys_dir.iterdir():
            if f"run{run}" in patch_file.name and ecode.lower() in patch_file.name:
                rep_dict[ecode] = patch_file
    files_list.append(rep_dict)

In [ ]:
files_metadata = build_wcp_metadata(cell_id=cell_name, 
                                    files_list=files_list, 
                                    ecode_timings=ecodes_cell_timings, 
                                    repetition_as_different_cells=False)
pprint(files_metadata[cell_name])

In [ ]:
cells = read_recordings(
    files_metadata=files_metadata,
    recording_reader=wcp_reader
)

In [ ]:
# define target features for different protocols
targets = get_ecode_targets(ecodes_cell_timings)

In [ ]:
targets

In [ ]:
global_tolerance = 30
for target in targets:
    target["tolerance"] = global_tolerance

In [ ]:
pprint(targets)

In [ ]:
t_start = time.time()
extract_efeatures_at_targets(
    cells, 
    targets,
    efel_settings={'interp_step': 0.1,
                   'strict_stiminterval': True}
)
t_stop = time.time()
print(f"Elapsed time {t_stop - t_start}")

In [ ]:
if cell_name == "cell1_211006_3148":
    majority = 0.4
else:
    majority = 0.2

In [ ]:
compute_rheobase(
    cells,
    protocols_rheobase=['IDthres'],
    rheobase_strategy="majority",
    rheobase_settings={"majority": majority}
)

In [ ]:
print(f"Cell rheobase: {cells[0].rheobase} nA")

In [ ]:
protocols = group_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=efeatures_output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

In [ ]:
efeatures["IDrest_300"]["soma"]

## 2) Convert to BPO format and append extra features


In [ ]:
from multimodalfitting.efeatures_extraction import compute_extra_features, convert_to_bpo_format,\
    append_extrafeatures_to_json

In [ ]:
protocols_of_interest = ["IDrest_150", "IDrest_250", "IDrest_300", "IV_-100", "IV_-20", "APWaveform_290"]

# remove sag features from IV_-20
exclude_features = {"IV_-20": ['sag_amplitude', 'sag_ratio1', 'sag_ratio2']}
# if cell_name == "cell1_211006_3148":
#     exclude_features.update({"IDrest_150": ['burst_number'],
#                              "IDrest_250": ['burst_number'],
#                              "IDrest_300": ['burst_number']})

in_protocol_path = efeatures_output_directory / "protocols.json"
in_efeatures_path = efeatures_output_directory / "features.json"

## Threshold EAP and extract featrues

In [ ]:
thresh_uV = 5

In [ ]:
amp_eap = np.ptp(eap, 1)
above_tr = np.where(amp_eap > thresh_uV)
eap_above = eap[above_tr]
probe_info = probe.info
locations_above = locations[above_tr]
probe_info["pos"] = locations_above.tolist()
probe_above = mu.return_mea(info=probe_info)

In [ ]:
fig_t, ax_t = plt.subplots(figsize=(10, 10))
ax_t = mu.plot_mea_recording(eap, probe, vscale=150, ax=ax_t)

In [ ]:
min_chan_above, _ = np.unravel_index(np.argmin(eap_above), eap_above.shape)

In [ ]:
second_chan_loc = np.array([-85, 50])
distances_sec = [np.linalg.norm(second_chan_loc - loc) for loc in locations_above[:, :2]]
second_chan_idx = np.argmin(distances_sec)

In [ ]:
fig_max, axs = plt.subplots(nrows=2, sharex=True) 
axs[0].plot(eap_above[min_chan_above, 15:80], color="C0", lw=3)
axs[1].plot(eap_above[second_chan_idx, 15:80], color="C1", lw=3)
axs[0].axis("off")
axs[1].axis("off")

In [ ]:
fig_t_max, ax_t = plt.subplots(figsize=(10, 10))
ax_t = mu.plot_mea_recording(eap_above, probe_above, vscale=150, ax=ax_t)
ax_t = mu.plot_mea_recording(eap_above, probe_above, channels=[min_chan_above],  colors="C0",
                             vscale=150, ax=ax_t, lw=2)
ax_t = mu.plot_mea_recording(eap_above, probe_above, channels=[second_chan_idx],  colors="C1",
                             vscale=150, ax=ax_t, lw=2)

In [ ]:
if save_fig:
    fig_t.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/template_above.pdf", dpi=300, transparent=True)
    fig_max.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/template_max.pdf", dpi=300, transparent=True)
    fig_t_max.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/template_above_max.pdf", dpi=300, transparent=True)

In [ ]:
# save thresholded probe_above.json and template
json.dump(probe_above.info, (efeatures_output_directory / "probe_BPO.json").open("w"))
np.save(efeatures_output_directory / "template_BPO.npy", eap_above)

In [ ]:
extra_features = compute_extra_features(eap_above, fs, upsample=10)

In [ ]:
pprint(extra_features.keys())

In [ ]:
fig_p2t = mf.plot_feature_map_w_colorbar(extra_features["peak_trough_ratio"], probe_above, 
                                         feature_name="Peak to trough ratio", label="a.u.", 
                                         cmap="coolwarm")
fig_negdiff = mf.plot_feature_map_w_colorbar(extra_features["neg_peak_diff"] * 1000, probe_above, 
                                             feature_name="Negative peak time difference", label="ms",
                                             cmap="PRGn")

In [ ]:
if save_fig:
    fig_p2t.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/p2t_ratio.pdf", transparent=True)
    fig_negdiff.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/neg_diff.pdf", transparent=True)
    fig_p2t.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/p2t_ratio.png", dpi=300, transparent=True)
    fig_negdiff.savefig("/Users/abuccino/Documents/Submissions/papers/multimodal/neg_diff.png", dpi=300, transparent=True)

### 3.1) "soma" features

In [ ]:
in_efeatures = json.load(open(in_efeatures_path, 'r'))

In [ ]:
out_protocol_path_soma = efeatures_output_directory / "protocols_BPO_soma.json"
out_efeatures_path_soma = efeatures_output_directory / "features_BPO_soma.json"

In [ ]:
protocols_dict_soma, efeatures_dict_soma = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                 out_protocol_path_soma, 
                                                                 out_efeatures_path_soma, 
                                                                 protocols_of_interest=protocols_of_interest, 
                                                                 exclude_features=exclude_features,
                                                                 std_from_mean=None)

### 3.2) "all" strategy

In [ ]:
out_protocol_path_all = efeatures_output_directory / "protocols_BPO_all.json"
out_efeatures_path_all = efeatures_output_directory / "features_BPO_all.json"

In [ ]:
protocols_dict_all, efeatures_dict_all = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                               out_protocol_path_all, 
                                                               out_efeatures_path_all, 
                                                               protocols_of_interest=protocols_of_interest, 
                                                               exclude_features=exclude_features,
                                                               std_from_mean=None)

In [ ]:
channel_ids_all = None

In [ ]:
extrafeatures_dict_all = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                      efeatures_dict=efeatures_dict_all,
                                                      efeatures_path=out_efeatures_path_all, 
                                                      channel_ids=None,
                                                      single_channel_features=False, 
                                                      std_from_mean=None)

In [ ]:
pprint(extrafeatures_dict_all["IDrest_300"]["MEA"])

### 3.3) "single" strategy

In [ ]:
cell = mf.create_experimental_model(model_name=model_name,
                                    release=True)
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe_above, 
                                     mechanisms_directory=model_folder)

In [ ]:
out_protocol_path_single = efeatures_output_directory / "protocols_BPO_single.json"
out_efeatures_path_single = efeatures_output_directory / "features_BPO_single.json"

In [ ]:
protocols_dict_single, efeatures_dict_single = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                     out_protocol_path_single, 
                                                                     out_efeatures_path_single, 
                                                                     protocols_of_interest=protocols_of_interest,
                                                                     exclude_features=exclude_features,
                                                                     std_from_mean=None)

In [ ]:
# ax_eap = mu.plot_mea_recording(eap_above / np.max(np.abs(eap_above), axis=1, keepdims=True), probe_above)

In [ ]:
# channel_ids_single = mf.select_single_channels(cell, sim, probe_above)
channel_ids_single = [186, 191, 84, 97, 68, 155, 203]

In [ ]:
channel_ids_single

In [ ]:
std_from_mean = 0.05

In [ ]:
extrafeatures_dict_single = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                         efeatures_dict=efeatures_dict_single,
                                                         efeatures_path=out_efeatures_path_single, 
                                                         channel_ids=channel_ids_single,
                                                         single_channel_features=True, 
                                                         std_from_mean=std_from_mean)

### 3.4) "sections" strategy

In [ ]:
out_protocol_path_sections = efeatures_output_directory / "protocols_BPO_sections.json"
out_efeatures_path_sections = efeatures_output_directory / "features_BPO_sections.json"

In [ ]:
protocols_dict_sections, efeatures_dict_sections = convert_to_bpo_format(in_protocol_path, in_efeatures_path, 
                                                                         out_protocol_path_sections, 
                                                                         out_efeatures_path_sections, 
                                                                         protocols_of_interest=protocols_of_interest,
                                                                         exclude_features=exclude_features,
                                                                         std_from_mean=None)

In [ ]:
# ax_eap = mu.plot_mea_recording(eap_above / np.max(np.abs(eap_above), axis=1, keepdims=True), probe_above)

In [ ]:
# channel_ids_sections = mf.select_mea_sections(cell, sim, probe_above)
channel_ids_sections = [
    [4,15, 20, 42, 48, 60, 99, 127, 130, 138, 145, 150, 157, 161, 183, 186],
    [0, 11, 12, 24, 35, 54, 72, 74, 80, 81, 83,  84, 94, 110, 114, 121, 128, 129, 133, 203, 207],
    [10, 14, 21, 26, 58, 67, 68, 70, 79, 87, 93, 131, 153, 155, 165, 182, 199],
    [5, 6, 29, 46, 77, 78, 125, 132, 171, 205]
]

In [ ]:
channel_ids_sections

In [ ]:
extrafeatures_dict_sections = append_extrafeatures_to_json(extra_features, protocol_name="IDrest_300",
                                                           efeatures_dict=efeatures_dict_sections,
                                                           efeatures_path=out_efeatures_path_sections, 
                                                           channel_ids=channel_ids_sections,
                                                           single_channel_features=False, 
                                                           std_from_mean=None)

In [ ]:
pprint(extrafeatures_dict_sections["IDrest_300"]["MEA"])

### Plot selected channels

In [ ]:
from matplotlib.patches import Ellipse, Rectangle

In [ ]:
fig_single, ax_single = plt.subplots()

for pos in probe.positions:
    rect = Rectangle(xy=pos[:2] - probe.info["size"], width=probe.info["size"][0],
                     height=probe.info["size"][1], edgecolor=f"orange",
                     fc=f"orange", lw=1, alpha=0.7)
    ax_single.add_patch(rect)


mf.plot_cell(cell, sim, detailed=False, exclude_sections=["soma"], ax=ax_single)
ellipse = Ellipse(xy=(0, 0), width=15, height=15, 
                  edgecolor="grey", color="grey", lw=2)
ax_single.add_patch(ellipse)

for i, chan in enumerate(channel_ids_single):
    rect = Rectangle(xy=probe_above.positions[chan, :2] - probe.info["size"], width=probe.info["size"][0],
                     height=probe.info["size"][1], edgecolor=f"C{i}",
                     fc=f"C{i}", lw=2, label=f"Channel {chan}", alpha=1)
    ax_single.add_patch(rect)
ax_single.legend()
ax_single.set_xlim(-300, 300)
ax_single.set_ylim(-320, 320)

In [ ]:
fig_sections, ax_sections = plt.subplots()

for pos in probe.positions:
    rect = Rectangle(xy=pos[:2] - probe.info["size"], width=probe.info["size"][0],
                     height=probe.info["size"][1], edgecolor=f"orange",
                     fc=f"orange", lw=1, alpha=0.7)
    ax_sections.add_patch(rect)
    
mf.plot_cell(cell, sim, detailed=False, exclude_sections=["soma"], ax=ax_sections)
ellipse = Ellipse(xy=(0, 0), width=15, height=15, 
                  edgecolor="grey", color="grey", lw=2)
ax_sections.add_patch(ellipse)

for i, chan_sect in enumerate(channel_ids_sections):
    for c, chan in enumerate(chan_sect):
        if c == 0:
            label = f"Section {i}"
        else:
            label = None
        rect = Rectangle(xy=probe_above.positions[chan, :2] - probe.info["size"], width=probe.info["size"][0],
                         height=probe.info["size"][1], edgecolor=f"C{i}",
                         fc=f"C{i}", lw=1, label=label)
        ax_sections.add_patch(rect)
ax_sections.legend()
ax_sections.set_xlim(-300, 300)
ax_sections.set_ylim(-320, 320)

In [ ]:
fig_single.savefig(f"single_{cell_name}.pdf")
fig_sections.savefig(f"sections_{cell_name}.pdf")